# 6. Resultados. Teste do melhor modelo

### Instalar/Importar bibliotecas

In [1]:
import pandas as pd
import os
import numpy as np

from pydantic import BaseModel
from typing import Optional, List

import joblib

### Carregar dados de entrada

Classe para ler dados de balanco patrimonial e demonstrativo financeiro

In [54]:
class AquisicaoDadosFundamentos(BaseModel):
    balancos_dir: Optional[str] = "dados/"
    balancos_joblib_file: Optional[str] = "dados/fundamentos_teste.joblib"
    fund_by_code: Optional[dict] = {}
    codes: List[str] = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        # Lista os arquivos xls na pasta dados/balancos
        files = os.listdir(self.balancos_dir)

        # Para cada arquivo (empresa) de balanco guarda no dicionario com o codigo como chave
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
    def salvar_joblib(self):
        joblib.dump(self.fund_by_code, self.balancos_joblib_file)



## Pré tratamento dos dados

## Remocao de dados omissos/nulos

### Remover empresas com colunas diferentes da empresa referencia PETR4

In [5]:

def remover_empresa_colunas_diff(fundamentos_by_code):

    fundamentos = fundamentos_by_code.copy()
    tamanho_inicial = len(fundamentos)

    columns_ref = list(fundamentos["ABEV3"].columns)
    """ 
    Remove empresas que nao tenham colunas de acordo com colunas de acao referencia 
    """
    codes = fundamentos.keys()
    empresa_a_remover = []
    for code in codes:
        if set(columns_ref) != set(fundamentos[code].columns):
            empresa_a_remover.append(code)
    
    for empresa in empresa_a_remover:
        fundamentos.pop(empresa)
    
    print((tamanho_inicial - len(fundamentos)), "empresas removidas por terem colunas diferentes de ABEV3")

    return fundamentos




### Remover colunas que contem algum tipo de dado nulo ou zerado.

Algumas colunas aqui tambem foi verificado que eram dados repetidos

In [6]:
def remover_colunas_nulas(fundamentos_by_code):
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()

    colunas_remover = ["Receita Bruta de Vendas e/ou Serviços",
                        "Ativos Biológicos",
                        "Despesas Antecipadas",
                        "Deduções da Receita Bruta",
                        "Resultado Não Operacional",
                        "Receitas",
                        "Despesas",
                        "Diferido",
                        "Adiantamento para Futuro Aumento Capital",
                        "Passivos sobre Ativos Não-Correntes a Venda e Descontinuados",
                        "Lucros e Receitas a Apropriar",
                        "Reservas de Reavaliação",
                        "Adiantamento para Futuro Aumento Capital",
                        "Perdas pela Não Recuperabilidade de Ativos",
                        "Outras Receitas Operacionais",
                        "Participações/Contribuições Estatutárias",
                        "Reversão dos Juros sobre Capital Próprio"]
    for code in codes:
        try:
            for col in colunas_remover:
                if col in fundamentos[code].columns:
                    fundamentos[code].drop(columns=col, inplace=True)
            print(f"colunas nulas de {code} removidas com sucesso")
        except KeyError as e:
            print(f"empresa {code} sem as colunas a remover - err", code)
            raise e

    return fundamentos

### Remover empresas que mesmo apos remover colunas que nao serao utilizadas, contem dados nulos

In [18]:
def remover_empresas_com_valores_nulos(fundamentos_by_code):
    
    fundamentos = fundamentos_by_code.copy()

    columns = fundamentos["ABEV3"].columns

    empresa_with_null = []

    for empresa in fundamentos:
        tabela = fundamentos[empresa]
        for coluna in columns:
            qtde_vazios = pd.isna(tabela[coluna]).sum()
            if isinstance(qtde_vazios, np.int64):
                if qtde_vazios > 0:
                    empresa_with_null.append(empresa)

    # remove empresas com valores nulos, ignorando ABEV por ter nulo na ultima linha
    for empresa in set(empresa_with_null):
        print(f"empresa {empresa} removida por ter valores nulos")
        fundamentos.pop(empresa)
    
    return fundamentos

### Funcoes parte de transformacao de dados

In [8]:
def transformar_indic_em_porcentagem_delta(dfs_in):
    dfs = dfs_in.copy()

    for code in dfs.keys():
        fundamento = dfs[code]
        fundamento.sort_index()
        for coluna in fundamento.columns:
            if coluna != "decisao":
                condicoes = [
                    (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
                ]
                valores = [
                    -1,
                    1,
                    (abs(fundamento[coluna].shift(1)) - abs(fundamento[coluna]))
                                / abs(fundamento[coluna].shift(1)),
                    1,
                    -1,
                    1,
                ]
                fundamento[coluna] = (
                    np.select(condicoes, valores, default=fundamento[coluna]
                              / fundamento[coluna].shift(1) - 1)
                )
        dfs[code] = fundamento
    return dfs

In [69]:
def transpor_colunas(df):
    df = df.copy()
    
    df = df.T
    df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
    
    return df
    
def juntar_dataframes(dfs_in):
    copia_fund = dfs_in.copy()
    bd = pd.DataFrame()
    for code in copia_fund:
        copia_fund[code] = copia_fund[code][1:-1]
        copia_fund[code] = copia_fund[code].reset_index(drop=True)
        bd = bd.append(copia_fund[code])
    bd.reset_index(drop=True, inplace=True)
    print(f"Dataframe unico criado com sucesso. Registros: {len(bd)}")
    return bd

In [93]:
def rename_columns(df):
    df = df.copy()
    columns_rename={
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Outros Ativos Circulantes": "outros_ativos_circ", 
        "Despesas Gerais e Administrativas": "despesas_gerais", 
        "Resultado Antes Tributação/Participações": "resultado_antes_trib"}
    
    for col, _ in columns_rename.items():
        if col not in list(df.columns):
            raise ValueError(f"coluna '{col}' nao presente no df.\n {[col for col in list(df.columns)]}")

    df.rename(columns=columns_rename, inplace=True)
            
    df.iloc[:,:-1] = df.iloc[:,:-1].astype(float)
    return df

### Feature selection

In [107]:
def remover_variaveis_ja_explicadas(df, manter_variaveis = None):
    manter_variaveis = ['outros_ativos_circ', 'receita_liq', 'despesas_gerais', 'resultado_antes_trib', 'lucro_op']
    df = df.loc[:, manter_variaveis]

    return df

### Predicao com melhor modelo

In [20]:
def carregar_modelo_ml():
    file_name: str = "../../out/result_ml_best_model.joblib"
    model = joblib.load(file_name)
    return model

In [118]:
def predict_decisao(modelo, fundamentos):
    for code, df in fundamentos.items():
        X = df.iloc[1:,:]
        y_predicted = modelo.predict(X)
        valor_pred = f"{y_predicted[0]}".replace('0', "Vender").replace('1', "Comprar")
        
        print(f"Decisao recomendada pelo modelo para: {code} -> {valor_pred}")

### Salvar dataframe em joblib para proximo notebook

In [21]:
def salvar_dfs_in_joblib(df, file_name: str = "out/result_teste_final.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df unico criado com sucesso")


# Execucao

In [119]:
def main():
    """
        Aquisicao dos dados
    """

    aquisicao_fund = AquisicaoDadosFundamentos()
    fundamentos_by_code = aquisicao_fund.run()

    """
        Pre tratamento dos valores nulos e colunas
    """
    fundamentos_by_code = remover_empresa_colunas_diff(fundamentos_by_code)
    fundamentos_by_code = remover_colunas_nulas(fundamentos_by_code)
    fundamentos_by_code = remover_empresas_com_valores_nulos(fundamentos_by_code)

    print(f"Quantidade final de empresas: {len(fundamentos_by_code)}")

    """
        Transformacao dos dados
    """
    for code, df in fundamentos_by_code.items():
        df_ = transpor_colunas(df)
        df_ = rename_columns(df_)
        df_ = remover_variaveis_ja_explicadas(df_)
        fundamentos_by_code[code] = df_

    fundamentos_by_code = transformar_indic_em_porcentagem_delta(fundamentos_by_code)

    """
        Predicao com melhor modelo
    """
    modelo = carregar_modelo_ml()
    
    predict_decisao(modelo, fundamentos_by_code)

    # # salvar em arquivo joblib
    # salvar_dfs_in_joblib(fundamentos_by_code)

    return fundamentos_by_code
    
fund = main()

carregando joblib fundamentos
0 empresas removidas por terem colunas diferentes de ABEV3
colunas nulas de CVCB3 removidas com sucesso
colunas nulas de ABEV3 removidas com sucesso
colunas nulas de COGN3 removidas com sucesso
colunas nulas de MOVI3 removidas com sucesso
Quantidade final de empresas: 4
Decisao recomendada pelo modelo para: CVCB3 -> Vender
Decisao recomendada pelo modelo para: ABEV3 -> Comprar
Decisao recomendada pelo modelo para: COGN3 -> Comprar
Decisao recomendada pelo modelo para: MOVI3 -> Comprar
